In [43]:
import os
import backtrader as bt

from btgym import BTgymEnv, BTgymDataset
from btgym.strategy.observers import Reward, Position, NormPnL
from btgym.algorithms import Launcher, A3C

from btgym.research import DevStrat_4_6

from btgym.algorithms.policy import Aac1dPolicy

In [48]:
MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    DevStrat_4_6,
    drawdown_call=5, # max % to loose, in percent of initial cash
    target_call=10,  # max % to win, same
    skip_frame=10,
)
# Set leveraged account:
MyCerebro.broker.setcash(100000)
brokerageCom = ((0.0001 +0.0000325)*0.18) + (0.0001 +0.0000325) + 0.00025 + 0.00002
MyCerebro.broker.setcommission(commission=brokerageCom, leverage=10.0) # commisssion to imitate spread
MyCerebro.addsizer(bt.sizers.SizerFix, stake=5000,) 
MyCerebro.broker.set_shortcash(False)

#MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Visualisations for reward, position and PnL dynamics:
MyCerebro.addobserver(Reward)
MyCerebro.addobserver(Position)
MyCerebro.addobserver(NormPnL)

MyDataset = BTgymDataset(
    #filename='.data/DAT_ASCII_EURUSD_M1_201703.csv',
    filename='btgym/examples/data/GMRINFRA.csv',
#     filename='./data/testsine_1min_period256_delta0002.csv',
    start_weekdays={0, 1, 2, 3},
    episode_duration={'days': 0, 'hours': 23, 'minutes': 55},
    start_00=False,
    time_gap={'hours': 6},
)


In [49]:
env_config = dict(
    class_ref=BTgymEnv,
    kwargs=dict(
        dataset=MyDataset,
        engine=MyCerebro,
        render_modes=['episode', 'human','external'],
        render_state_as_image=True,
        render_ylabel='OHL_diff.',
        render_size_episode=(12,8),
        render_size_human=(9, 4),
        render_size_state=(11, 3),
        render_dpi=75,
        port=5000,
        data_port=4999,
        verbose=0,  # better be 0
    )
)

cluster_config = dict(
    host='127.0.0.1',
    port=12230,
    num_workers=1,  # Set according CPU's available 
    num_ps=1,
    num_envs=1,  # do not change yet
    log_dir=os.path.expanduser('~/tmp/test_gym_a3c'),
)

policy_config = dict(
    class_ref=Aac1dPolicy,
    kwargs={}
)

trainer_config = dict(
    class_ref=A3C,
    kwargs=dict(
        opt_learn_rate=[1e-4, 1e-4], # or random log-uniform range, values > 2e-4 can ruin training 
        opt_end_learn_rate=1e-5,
        opt_decay_steps=100*10**6,
        model_gamma=0.95,
        model_gae_lambda=1.0,
        model_beta=0.01, # Entropy reg
        rollout_length=20,
        time_flat=False,
        model_summary_freq=100,
        episode_summary_freq=5,
        env_render_freq=20,
    )
)

In [51]:
launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    trainer_config=trainer_config,
    policy_config=policy_config,
    test_mode=False,
    max_env_steps=100*10**6,
    root_random_seed=0,
    purge_previous=1,  # ask to override previously saved model and logs
    verbose=1  # 0 or 1
)

# Train it:
launcher.run()

[2018-02-26 17:56:52.458984] INFO: LauncherShell: Random seed: 0
</home/s2c/tmp/test_gym_a3c> already exists. Override[y/n]? y
[2018-02-26 17:56:54.379271] NOTICE: LauncherShell: Files in </home/s2c/tmp/test_gym_a3c> purged.
[2018-02-26 17:56:54.884339] INFO: BTgymAPIshell_0: Custom Dataset class used.
[2018-02-26 17:56:54.885125] INFO: BTgymAPIshell_0: Connecting data_server...
[2018-02-26 17:56:54.934572] INFO: BTgymDataServer_0: PID: 28566
[2018-02-26 17:56:55.092253] WARNING: SimpleDataSet_0: Found 1500 duplicated date_time records in <btgym/examples/data/GMRINFRA.csv>.                     Removed all but first occurrences.
[2018-02-26 17:56:55.092925] INFO: SimpleDataSet_0: Loaded 14250 records from <btgym/examples/data/GMRINFRA.csv>.
[2018-02-26 17:56:55.105763] INFO: SimpleDataSet_0: Data summary:
               open          high           low         close        volume
count  14250.000000  14250.000000  14250.000000  14250.000000  1.425000e+04
mean      21.025211     21.06298

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession


[2018-02-26 17:56:59.138276] INFO: Worker_0: connecting to the parameter server... 
[2018-02-26 17:56:59.307600] INFO: Worker_0: initializing all parameters.

********************************************************************************************
**  Press `Ctrl-C` or jupyter:[Kernel]->[Interrupt] to stop training and close launcher.  **
********************************************************************************************

INFO:tensorflow:Starting standard services.


INFO:tensorflow:Starting standard services.


INFO:tensorflow:Saving checkpoint to path /home/s2c/tmp/test_gym_a3c/train/model.ckpt


...

[2018-02-26 17:58:00.742866] ERROR: BTgymServer_0: BtgymServer_sampling_attempt: data_server unreachable with status: <receive_failed_due_to_connect_timeout>.


Process BTgymServer-57:2:
Traceback (most recent call last):
  File "/home/s2c/anaconda3/envs/AlgoTrading/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/s2c/go/src/kite-go/btgym/btgym/server.py", line 498, in run
    trial_sample, trial_stat, dataset_stat = self.get_data(**sample_config['trial_config'])
  File "/home/s2c/go/src/kite-go/btgym/btgym/server.py", line 305, in get_data
    raise ConnectionError(msg)
ConnectionError: BtgymServer_sampling_attempt: data_server unreachable with status: <receive_failed_due_to_connect_timeout>.


[2018-02-26 17:58:00.778611] ERROR: BTgymAPIshell_0: .step(): server unreachable with status: <receive_failed_due_to_connect_timeout>.
[2018-02-26 17:58:00.779290] ERROR: ThreadRunner_0: RunTime exception occurred.

Press `Ctrl-C` or jupyter:[Kernel]->[Interrupt] for clean exit.

Traceback (most recent call last):
  File "/home/s2c/go/src/kite-go/btgym/btgym/algorithms/runner/threadrunner.py", line 91, in run
    self._run()
  File "/home/s2c/go/src/kite-go/btgym/btgym/algorithms/runner/threadrunner.py", line 118, in _run
    self.queue.put(next(rollout_provider), timeout=600.0)
  File "/home/s2c/go/src/kite-go/btgym/btgym/algorithms/runner/base.py", line 45, in BaseEnvRunnerFn
    last_state = env.reset(**policy.get_sample_config())
  File "/home/s2c/go/src/kite-go/btgym/btgym/envs/backtrader.py", line 667, in reset
    self.env_response = self.step(0)
  File "/home/s2c/go/src/kite-go/btgym/btgym/envs/backtrader.py", line 753, in step
    raise ConnectionError(msg)
ConnectionError: .s

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/s2c/go/src/kite-go/btgym/btgym/algorithms/runner/threadrunner.py", line 91, in run
    self._run()
  File "/home/s2c/go/src/kite-go/btgym/btgym/algorithms/runner/threadrunner.py", line 118, in _run
    self.queue.put(next(rollout_provider), timeout=600.0)
  File "/home/s2c/go/src/kite-go/btgym/btgym/algorithms/runner/base.py", line 45, in BaseEnvRunnerFn
    last_state = env.reset(**policy.get_sample_config())
  File "/home/s2c/go/src/kite-go/btgym/btgym/envs/backtrader.py", line 667, in reset
    self.env_response = self.step(0)
  File "/home/s2c/go/src/kite-go/btgym/btgym/envs/backtrader.py", line 753, in step
    raise ConnectionError(msg)
ConnectionError: .step(): server unreachable with status: <receive_failed_due_to_connect_timeout>.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/s2c/anaconda3/envs/AlgoTrading/lib/python3.6/threadi

[2018-02-26 17:58:52.524081] NOTICE: LauncherShell: chief_worker_0 has joined.
[2018-02-26 17:58:52.524680] NOTICE: LauncherShell: parameter_server_0 has joined.
[2018-02-26 17:58:52.525136] NOTICE: LauncherShell: Launcher closed.
